In [ ]:
'''
import os
import shutil

source_dir = "masklab"  
destination_dir = "masklab_jpg_only"  

if not os.path.exists(destination_dir):
    os.makedirs(destination_dir)

for subdir, dirs, files in os.walk(source_dir):
    for file in files:
        if file.lower().endswith(".jpg"):
            rel_path = os.path.relpath(subdir, source_dir)
            dest_subdir = os.path.join(destination_dir, rel_path)
            if not os.path.exists(dest_subdir):
                os.makedirs(dest_subdir)

            src_file_path = os.path.join(subdir, file)
            dest_file_path = os.path.join(dest_subdir, file)
            shutil.copy(src_file_path, dest_file_path)'
'''

In [ ]:
import os
import shutil

dir = "masklab_jpg_only"

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd

model_name = "mymodel_echocv_500-500-8_adam_16_0.9394.h5"
input_shape = (224,224,3)
batch_size = 2

test_datagen = ImageDataGenerator()
test_generator = test_datagen.flow_from_directory(
    directory=dir,
    target_size=input_shape[:2],
    color_mode='rgb',
    class_mode=None,
    batch_size=batch_size,
    shuffle=False
)
STEP_SIZE_TEST = test_generator.n/batch_size
test_generator.reset()

model = load_model(model_name)
pred = model.predict_generator(test_generator, verbose=1)

confidence = ["{0:.3f}".format(np.amax(p)) for p in pred]
predicted_class_indices = np.argmax(pred,axis=1)
labels = { 'plax':0, 'psax-av':1, 'psax-mv':2, 'psax-ap':3, 'a4c':4, 'a5c':5, 'a3c':6, 'a2c':7 }
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
filenames=test_generator.filenames
results = pd.DataFrame({"Filename":filenames, "Prediction":predictions, "Confidence":confidence})
results_file = "masklab_results.csv"
results.to_csv(results_file, index=False, encoding='utf-8-sig')